In [1]:
# Importing Libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [3]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [4]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [5]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [6]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [7]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [8]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [9]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [10]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [11]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [12]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [13]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [14]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [15]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [24]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [26]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 85s 12ms/step - loss: 1.3786 - acc: 0.4440 - val_loss: 1.2528 - val_acc: 0.4235
Epoch 2/30
7352/7352 [==============================] - 81s 11ms/step - loss: 1.0784 - acc: 0.5352 - val_loss: 1.0303 - val_acc: 0.5752
Epoch 3/30
7352/7352 [==============================] - 81s 11ms/step - loss: 0.8563 - acc: 0.6166 - val_loss: 0.8010 - val_acc: 0.6125
Epoch 4/30
7352/7352 [==============================] - 81s 11ms/step - loss: 0.7799 - acc: 0.6283 - val_loss: 0.8730 - val_acc: 0.5660
Epoch 5/30
7352/7352 [==============================] - 81s 11ms/step - loss: 0.8888 - acc: 0.6026 - val_loss: 0.8774 - val_acc: 0.5996
Epoch 6/30
7352/7352 [==============================] - 81s 11ms/step - loss: 0.8521 - acc: 0.6118 - val_loss: 0.7886 - val_acc: 0.6715
Epoch 7/30
7352/7352 [==============================] - 81s 11ms/step - loss: 0.7086 - acc: 0.6759 - val_loss: 0.7724 - val_acc: 0.6719


In [27]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0        0                   0   
SITTING                 11      360        95        2                   6   
STANDING                 0       78       448        4                   0   
WALKING                  0        0         0      457                  33   
WALKING_DOWNSTAIRS       0        0         0       21                 397   
WALKING_UPSTAIRS         0        0         1       63                   8   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                           17  
STANDING                           2  
WALKING                            6  
WALKING_DOWNSTAIRS                 2  
WALKING_UPSTAIRS                 399  


In [28]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 2s 824us/step


In [29]:
score

[0.8186306150371352, 0.8724126230064473]

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning

### More Dropout Rate and RELU Activation

In [25]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.7))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [27]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 37s 5ms/step - loss: 1.4339 - acc: 0.4113 - val_loss: 1.2448 - val_acc: 0.4595
Epoch 2/30
7352/7352 [==============================] - 34s 5ms/step - loss: 1.1768 - acc: 0.4890 - val_loss: 1.1081 - val_acc: 0.4903
Epoch 3/30
7352/7352 [==============================] - 34s 5ms/step - loss: 1.0834 - acc: 0.5320 - val_loss: 1.0997 - val_acc: 0.5232
Epoch 4/30
7352/7352 [==============================] - 34s 5ms/step - loss: 1.0673 - acc: 0.5393 - val_loss: 1.1627 - val_acc: 0.4506
Epoch 5/30
7352/7352 [==============================] - 34s 5ms/step - loss: 1.2533 - acc: 0.4285 - val_loss: 1.2076 - val_acc: 0.3902
Epoch 6/30
7352/7352 [==============================] - 34s 5ms/step - loss: 1.2720 - acc: 0.4165 - val_loss: 1.3062 - val_acc: 0.3831
Epoch 7/30
7352/7352 [==============================] - 34s 5ms/step - loss: 1.2266 - acc: 0.4674 - val_loss: 1.0613 - val_acc: 0.5470
Epoch 8

In [28]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_UPSTAIRS
True                                                                    
LAYING                 532        0         0        5                 0
SITTING                  0      369       116        6                 0
STANDING                 1       88       427       13                 3
WALKING                  0        0         2      463                31
WALKING_DOWNSTAIRS       2        0         1      343                74
WALKING_UPSTAIRS         0        0         2      373                96


### Less Dropout Rate

In [29]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.3))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [31]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 83s 11ms/step - loss: 1.2783 - acc: 0.4527 - val_loss: 1.0790 - val_acc: 0.5314
Epoch 2/30
7352/7352 [==============================] - 79s 11ms/step - loss: 0.8672 - acc: 0.6559 - val_loss: 0.8655 - val_acc: 0.6400
Epoch 3/30
7352/7352 [==============================] - 79s 11ms/step - loss: 0.7116 - acc: 0.7038 - val_loss: 0.7483 - val_acc: 0.6807
Epoch 4/30
7352/7352 [==============================] - 79s 11ms/step - loss: 0.6267 - acc: 0.7231 - val_loss: 0.7348 - val_acc: 0.7017
Epoch 5/30
7352/7352 [==============================] - 79s 11ms/step - loss: 0.5954 - acc: 0.7458 - val_loss: 0.7752 - val_acc: 0.6756
Epoch 6/30
7352/7352 [==============================] - 79s 11ms/step - loss: 0.5434 - acc: 0.7816 - val_loss: 0.7116 - val_acc: 0.7078
Epoch 7/30
7352/7352 [==============================] - 80s 11ms/step - loss: 0.4661 - acc: 0.8137 - val_loss: 0.6520 - val_acc: 0.7421


In [32]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        27        0                   0   
SITTING                  1      389        98        1                   0   
STANDING                 0       84       448        0                   0   
WALKING                  0        0         0      491                   5   
WALKING_DOWNSTAIRS       0        0         0        8                 407   
WALKING_UPSTAIRS         0        0         5       42                  25   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            2  
STANDING                           0  
WALKING                            0  
WALKING_DOWNSTAIRS                 5  
WALKING_UPSTAIRS                 399  


### More Number of layers

In [33]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(48, input_shape=(timesteps, input_dim), return_sequences=True))
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(LSTM(32))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 128, 48)           11136     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128, 48)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                10368     
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 198       
Total params: 21,702
Trainable params: 21,702
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [35]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 173s 24ms/step - loss: 1.0254 - acc: 0.5683 - val_loss: 0.8267 - val_acc: 0.6440
Epoch 2/30
7352/7352 [==============================] - 168s 23ms/step - loss: 0.6917 - acc: 0.6812 - val_loss: 0.8010 - val_acc: 0.6590
Epoch 3/30
7352/7352 [==============================] - 166s 23ms/step - loss: 0.4841 - acc: 0.8164 - val_loss: 0.5118 - val_acc: 0.8195
Epoch 4/30
7352/7352 [==============================] - 168s 23ms/step - loss: 0.3006 - acc: 0.9030 - val_loss: 0.4843 - val_acc: 0.8398
Epoch 5/30
7352/7352 [==============================] - 167s 23ms/step - loss: 0.1959 - acc: 0.9325 - val_loss: 0.3837 - val_acc: 0.8741
Epoch 6/30
7352/7352 [==============================] - 167s 23ms/step - loss: 0.1873 - acc: 0.9368 - val_loss: 0.3566 - val_acc: 0.8962
Epoch 7/30
7352/7352 [==============================] - 167s 23ms/step - loss: 0.1600 - acc: 0.9433 - val_loss: 0.3318 - val_acc: 

In [36]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  5      281       203        0                   1   
STANDING                 0       33       496        3                   0   
WALKING                  0        0         1      465                  16   
WALKING_DOWNSTAIRS       0        0         0        4                 406   
WALKING_UPSTAIRS         0        0         2       12                   6   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            1  
STANDING                           0  
WALKING                           14  
WALKING_DOWNSTAIRS                10  
WALKING_UPSTAIRS                 451  


In [155]:
from keras.layers import Conv2D, MaxPooling2D, MaxPooling1D
from keras.layers import Conv1D, MaxPooling3D, Embedding, Flatten


In [71]:
timesteps

128

In [100]:
model = Sequential()
model.add(Conv1D(180, 3, input_shape=(timesteps, input_dim), activation='relu'))
model.add(Conv1D(64, 3, activation='sigmoid'))
# model.add(Dropout(0.70))
model.add(Conv1D(100, 3, activation='relu'))
model.add(Conv1D(100, 3, activation='relu'))
model.add(Conv1D(48, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(6, activation='softmax'))
# model.add(Dropout(0.70))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy']) 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_106 (Conv1D)          (None, 126, 180)          5040      
_________________________________________________________________
conv1d_107 (Conv1D)          (None, 124, 64)           34624     
_________________________________________________________________
conv1d_108 (Conv1D)          (None, 122, 100)          19300     
_________________________________________________________________
conv1d_109 (Conv1D)          (None, 120, 100)          30100     
_________________________________________________________________
conv1d_110 (Conv1D)          (None, 118, 48)           14448     
_________________________________________________________________
flatten_23 (Flatten)         (None, 5664)              0         
_________________________________________________________________
dense_27 (Dense)             (None, 6)                 33990     
Total para

In [101]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=20)

Train on 7352 samples, validate on 2947 samples
Epoch 1/20
7352/7352 [==============================] - 58s 8ms/step - loss: 0.7124 - acc: 0.6748 - val_loss: 0.5101 - val_acc: 0.7889
Epoch 2/20
7352/7352 [==============================] - 51s 7ms/step - loss: 0.3312 - acc: 0.8848 - val_loss: 0.6078 - val_acc: 0.8595
Epoch 3/20
7352/7352 [==============================] - 51s 7ms/step - loss: 0.1751 - acc: 0.9306 - val_loss: 0.5268 - val_acc: 0.8860
Epoch 4/20
7352/7352 [==============================] - 53s 7ms/step - loss: 0.1730 - acc: 0.9338 - val_loss: 0.6330 - val_acc: 0.8714
Epoch 5/20
7352/7352 [==============================] - 53s 7ms/step - loss: 0.1547 - acc: 0.9404 - val_loss: 0.5233 - val_acc: 0.8870
Epoch 6/20
7352/7352 [==============================] - 53s 7ms/step - loss: 0.1311 - acc: 0.9465 - val_loss: 0.5823 - val_acc: 0.8887
Epoch 7/20
7352/7352 [==============================] - 53s 7ms/step - loss: 0.1189 - acc: 0.9490 - val_loss: 0.4969 - val_acc: 0.8982
Epoch 8

In [84]:
# model=Sequential()
# model.add(Conv2D(64, kernel_size=(5, 5),activation='relu', padding='same', input_shape=(timesteps, input_dim))) 
# model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides = 2)) 
# model.add(Dropout(0.5)) 
# model.add(Conv2D(32, (5, 5), activation='relu', padding='same')) 
# model.add(Conv2D(32, (5, 5), activation='relu', padding='same'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides = 2)) 
# model.add(BatchNormalization()) 
# model.add(Dropout(0.5)) 
# model.add(Flatten())
# model.add(Dense(100, activation='relu')) 
# model.add(BatchNormalization()) 
# model.add(Dropout(0.5)) 
# model.add(Dense(n_classes, activation='softmax')) 
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy']) 
# model.summary()

In [80]:
# # Compiling the model
# model.compile(loss='categorical_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

In [85]:
# # Training the model
# model.fit(X_train,
#           Y_train,
#           batch_size=batch_size,
#           validation_data=(X_test, Y_test),
#           epochs=20)

In [102]:
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0        0                   0   
SITTING                  0      403        60        0                   0   
STANDING                 0       96       434        0                   0   
WALKING                  0        0         0      492                   2   
WALKING_DOWNSTAIRS       0        0         0        7                 386   
WALKING_UPSTAIRS         0        0         0       14                  22   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                           28  
STANDING                           2  
WALKING                            2  
WALKING_DOWNSTAIRS                27  
WALKING_UPSTAIRS                 435  


In [103]:
model = Sequential()
model.add(Conv1D(180, 3, input_shape=(timesteps, input_dim), activation='relu'))
model.add(Conv1D(64, 3, activation='sigmoid'))

model.add(Conv1D(100, 3, activation='relu'))
model.add(Conv1D(100, 3, activation='relu'))
model.add(Dropout(0.70))
model.add(Conv1D(48, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(6, activation='softmax'))
# model.add(Dropout(0.70))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy']) 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_111 (Conv1D)          (None, 126, 180)          5040      
_________________________________________________________________
conv1d_112 (Conv1D)          (None, 124, 64)           34624     
_________________________________________________________________
conv1d_113 (Conv1D)          (None, 122, 100)          19300     
_________________________________________________________________
conv1d_114 (Conv1D)          (None, 120, 100)          30100     
_________________________________________________________________
dropout_35 (Dropout)         (None, 120, 100)          0         
_________________________________________________________________
conv1d_115 (Conv1D)          (None, 118, 48)           14448     
_________________________________________________________________
flatten_24 (Flatten)         (None, 5664)              0         
__________

In [104]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=20)

Train on 7352 samples, validate on 2947 samples
Epoch 1/20
7352/7352 [==============================] - 64s 9ms/step - loss: 0.8332 - acc: 0.6239 - val_loss: 0.6980 - val_acc: 0.7621
Epoch 2/20
7352/7352 [==============================] - 60s 8ms/step - loss: 0.3117 - acc: 0.8837 - val_loss: 0.6173 - val_acc: 0.8422
Epoch 3/20
7352/7352 [==============================] - 60s 8ms/step - loss: 0.2038 - acc: 0.9211 - val_loss: 0.4948 - val_acc: 0.8751
Epoch 4/20
7352/7352 [==============================] - 60s 8ms/step - loss: 0.2018 - acc: 0.9289 - val_loss: 0.4100 - val_acc: 0.8843
Epoch 5/20
7352/7352 [==============================] - 60s 8ms/step - loss: 0.1477 - acc: 0.9416 - val_loss: 0.4456 - val_acc: 0.9118
Epoch 6/20
7352/7352 [==============================] - 60s 8ms/step - loss: 0.1403 - acc: 0.9418 - val_loss: 0.3199 - val_acc: 0.9131
Epoch 7/20
7352/7352 [==============================] - 60s 8ms/step - loss: 0.1455 - acc: 0.9376 - val_loss: 0.3042 - val_acc: 0.9199
Epoch 8

In [105]:
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  0      424        63        1                   1   
STANDING                 0      106       426        0                   0   
WALKING                  0        0         0      471                   4   
WALKING_DOWNSTAIRS       0        0         0        1                 414   
WALKING_UPSTAIRS         0        0         0        4                  22   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            2  
STANDING                           0  
WALKING                           21  
WALKING_DOWNSTAIRS                 5  
WALKING_UPSTAIRS                 445  


In [107]:
model = Sequential()
model.add(Conv1D(180, 3, input_shape=(timesteps, input_dim), activation='relu'))
model.add(Conv1D(64, 3, activation='sigmoid'))

model.add(Conv1D(100, 3, activation='relu'))
model.add(Dropout(0.70))
model.add(Conv1D(100, 3, activation='relu'))
model.add(Dropout(0.70))
model.add(Conv1D(48, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(6, activation='softmax'))
# model.add(Dropout(0.70))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy']) 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_119 (Conv1D)          (None, 126, 180)          5040      
_________________________________________________________________
conv1d_120 (Conv1D)          (None, 124, 64)           34624     
_________________________________________________________________
conv1d_121 (Conv1D)          (None, 122, 100)          19300     
_________________________________________________________________
dropout_37 (Dropout)         (None, 122, 100)          0         
_________________________________________________________________
conv1d_122 (Conv1D)          (None, 120, 100)          30100     
_________________________________________________________________
dropout_38 (Dropout)         (None, 120, 100)          0         
_________________________________________________________________
conv1d_123 (Conv1D)          (None, 118, 48)           14448     
__________

In [108]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=20)

Train on 7352 samples, validate on 2947 samples
Epoch 1/20
7352/7352 [==============================] - 69s 9ms/step - loss: 1.0805 - acc: 0.5041 - val_loss: 0.6758 - val_acc: 0.7116
Epoch 2/20
7352/7352 [==============================] - 57s 8ms/step - loss: 0.5067 - acc: 0.7654 - val_loss: 1.3113 - val_acc: 0.5653
Epoch 3/20
7352/7352 [==============================] - 65s 9ms/step - loss: 0.3559 - acc: 0.8644 - val_loss: 0.6839 - val_acc: 0.7109
Epoch 4/20
7352/7352 [==============================] - 64s 9ms/step - loss: 0.2556 - acc: 0.9066 - val_loss: 0.7907 - val_acc: 0.7245
Epoch 5/20
7352/7352 [==============================] - 66s 9ms/step - loss: 0.2374 - acc: 0.9113 - val_loss: 0.8164 - val_acc: 0.7462
Epoch 6/20
7352/7352 [==============================] - 67s 9ms/step - loss: 0.2084 - acc: 0.9207 - val_loss: 0.7986 - val_acc: 0.6291
Epoch 7/20
7352/7352 [==============================] - 67s 9ms/step - loss: 0.1763 - acc: 0.9320 - val_loss: 0.9645 - val_acc: 0.6580
Epoch 8

In [109]:
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 518        0        19        0                   0   
SITTING                  0      154       328        0                   4   
STANDING                 0        0       525        0                   0   
WALKING                  0        0         0       23                 379   
WALKING_DOWNSTAIRS       0        0         0        0                 420   
WALKING_UPSTAIRS         0        0         0        0                 132   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            5  
STANDING                           7  
WALKING                           94  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 339  


In [110]:
model = Sequential()
model.add(Conv1D(180, 3, input_shape=(timesteps, input_dim), activation='relu'))
model.add(Conv1D(128, 3, activation='sigmoid'))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Dropout(0.50))
model.add(Conv1D(100, 3, activation='sigmoid'))
model.add(Dropout(0.70))
model.add(Conv1D(100, 3, activation='relu'))
model.add(Dropout(0.70))
model.add(Conv1D(48, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(6, activation='softmax'))
# model.add(Dropout(0.70))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy']) 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_124 (Conv1D)          (None, 126, 180)          5040      
_________________________________________________________________
conv1d_125 (Conv1D)          (None, 124, 128)          69248     
_________________________________________________________________
conv1d_126 (Conv1D)          (None, 122, 64)           24640     
_________________________________________________________________
dropout_39 (Dropout)         (None, 122, 64)           0         
_________________________________________________________________
conv1d_127 (Conv1D)          (None, 120, 100)          19300     
_________________________________________________________________
dropout_40 (Dropout)         (None, 120, 100)          0         
_________________________________________________________________
conv1d_128 (Conv1D)          (None, 118, 100)          30100     
__________

In [111]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=20)

Train on 7352 samples, validate on 2947 samples
Epoch 1/20
7352/7352 [==============================] - 94s 13ms/step - loss: 1.4955 - acc: 0.3467 - val_loss: 0.7858 - val_acc: 0.5999
Epoch 2/20
7352/7352 [==============================] - 89s 12ms/step - loss: 0.6623 - acc: 0.6797 - val_loss: 0.6423 - val_acc: 0.7621
Epoch 3/20
7352/7352 [==============================] - 95s 13ms/step - loss: 0.4233 - acc: 0.8236 - val_loss: 0.4580 - val_acc: 0.8297
Epoch 4/20
7352/7352 [==============================] - 94s 13ms/step - loss: 0.2981 - acc: 0.8844 - val_loss: 0.4995 - val_acc: 0.8045
Epoch 5/20
7352/7352 [==============================] - 94s 13ms/step - loss: 0.2476 - acc: 0.9051 - val_loss: 0.4902 - val_acc: 0.8551
Epoch 6/20
7352/7352 [==============================] - 93s 13ms/step - loss: 0.2052 - acc: 0.9188 - val_loss: 0.4677 - val_acc: 0.8476
Epoch 7/20
7352/7352 [==============================] - 67s 9ms/step - loss: 0.2146 - acc: 0.9236 - val_loss: 0.5333 - val_acc: 0.8531
E

In [112]:
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 509        0         0        3                  25   
SITTING                  0      428        53        4                   3   
STANDING                 0      115       417        0                   0   
WALKING                  0        0         0      411                   9   
WALKING_DOWNSTAIRS       0        0         0        0                 408   
WALKING_UPSTAIRS         0        0         0        1                  26   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            3  
STANDING                           0  
WALKING                           76  
WALKING_DOWNSTAIRS                12  
WALKING_UPSTAIRS                 444  


In [143]:
from keras.layers.normalization import BatchNormalization
model = Sequential()
model.add(Conv1D(180, 3, input_shape=(timesteps, input_dim), activation='relu'))
model.add(Conv1D(64, 3, activation='sigmoid'))

model.add(Conv1D(100, 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(100, 3, activation='relu'))
model.add(Dropout(0.70))
model.add(Conv1D(48, 3, activation='relu'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(6, activation='softmax'))
# model.add(Dropout(0.70))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy']) 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_258 (Conv1D)          (None, 126, 180)          5040      
_________________________________________________________________
conv1d_259 (Conv1D)          (None, 124, 64)           34624     
_________________________________________________________________
conv1d_260 (Conv1D)          (None, 122, 100)          19300     
_________________________________________________________________
batch_normalization_7 (Batch (None, 122, 100)          400       
_________________________________________________________________
conv1d_261 (Conv1D)          (None, 120, 100)          30100     
_________________________________________________________________
dropout_167 (Dropout)        (None, 120, 100)          0         
_________________________________________________________________
conv1d_262 (Conv1D)          (None, 118, 48)           14448     
__________

In [144]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=25)

Train on 7352 samples, validate on 2947 samples
Epoch 1/25
7352/7352 [==============================] - 44s 6ms/step - loss: 0.5258 - acc: 0.8183 - val_loss: 4.0344 - val_acc: 0.5103
Epoch 2/25
7352/7352 [==============================] - 45s 6ms/step - loss: 0.3225 - acc: 0.8973 - val_loss: 9.8705 - val_acc: 0.3482
Epoch 3/25
7352/7352 [==============================] - 45s 6ms/step - loss: 0.2328 - acc: 0.9159 - val_loss: 1.4637 - val_acc: 0.6834
Epoch 4/25
7352/7352 [==============================] - 45s 6ms/step - loss: 0.2205 - acc: 0.9211 - val_loss: 1.8193 - val_acc: 0.5989
Epoch 5/25
7352/7352 [==============================] - 45s 6ms/step - loss: 0.2224 - acc: 0.9212 - val_loss: 0.4619 - val_acc: 0.8537
Epoch 6/25
7352/7352 [==============================] - 49s 7ms/step - loss: 0.2087 - acc: 0.9251 - val_loss: 1.0139 - val_acc: 0.7672
Epoch 7/25
7352/7352 [==============================] - 41s 6ms/step - loss: 0.1769 - acc: 0.9328 - val_loss: 1.1607 - val_acc: 0.7326
Epoch 8

In [145]:
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                  27      239       271        0                   0   
SITTING                  0      430        40        0                   0   
STANDING                 0      135       396        0                   0   
WALKING                  0        1         1      476                  18   
WALKING_DOWNSTAIRS       0        0         0        0                 419   
WALKING_UPSTAIRS         0        8         1        1                  24   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                           21  
STANDING                           1  
WALKING                            0  
WALKING_DOWNSTAIRS                 1  
WALKING_UPSTAIRS                 437  


In [146]:
model = Sequential()
model.add(Conv1D(180, 3, input_shape=(timesteps, input_dim), activation='relu'))
model.add(Conv1D(64, 3, activation='sigmoid'))

model.add(Conv1D(100, 3, activation='relu'))
model.add(Conv1D(100, 3, activation='relu'))
model.add(Dropout(0.70))
model.add(Conv1D(48, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(6, activation='softmax'))
# model.add(Dropout(0.70))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy']) 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_263 (Conv1D)          (None, 126, 180)          5040      
_________________________________________________________________
conv1d_264 (Conv1D)          (None, 124, 64)           34624     
_________________________________________________________________
conv1d_265 (Conv1D)          (None, 122, 100)          19300     
_________________________________________________________________
conv1d_266 (Conv1D)          (None, 120, 100)          30100     
_________________________________________________________________
dropout_168 (Dropout)        (None, 120, 100)          0         
_________________________________________________________________
conv1d_267 (Conv1D)          (None, 118, 48)           14448     
_________________________________________________________________
flatten_32 (Flatten)         (None, 5664)              0         
__________

In [147]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=30)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 79s 11ms/step - loss: 0.8301 - acc: 0.6137 - val_loss: 0.6556 - val_acc: 0.7095
Epoch 2/30
7352/7352 [==============================] - 70s 10ms/step - loss: 0.3570 - acc: 0.8569 - val_loss: 0.7334 - val_acc: 0.8195
Epoch 3/30
7352/7352 [==============================] - 69s 9ms/step - loss: 0.2380 - acc: 0.9097 - val_loss: 0.6728 - val_acc: 0.8527
Epoch 4/30
7352/7352 [==============================] - 68s 9ms/step - loss: 0.2183 - acc: 0.9222 - val_loss: 0.6101 - val_acc: 0.8456
Epoch 5/30
7352/7352 [==============================] - 69s 9ms/step - loss: 0.1811 - acc: 0.9323 - val_loss: 0.4502 - val_acc: 0.8778
Epoch 6/30
7352/7352 [==============================] - 71s 10ms/step - loss: 0.1447 - acc: 0.9429 - val_loss: 0.7354 - val_acc: 0.8178
Epoch 7/30
7352/7352 [==============================] - 68s 9ms/step - loss: 0.1519 - acc: 0.9400 - val_loss: 0.3219 - val_acc: 0.8989
Epoc

In [148]:
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0       24                   0   
SITTING                  0      417        68        0                   6   
STANDING                 0      103       429        0                   0   
WALKING                  0        0         0      474                  21   
WALKING_DOWNSTAIRS       0        0         0        2                 418   
WALKING_UPSTAIRS         0        0         0        0                  36   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             3  
SITTING                            0  
STANDING                           0  
WALKING                            1  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 435  


In [189]:
from keras.optimizers import Adam

In [190]:
model = Sequential()
model.add(Conv1D(180, 3, input_shape=(timesteps, input_dim), activation='relu'))
model.add(Dropout(0.70))
model.add(Conv1D(128, 3, activation='sigmoid'))
model.add(Dropout(0.70))
model.add(Conv1D(100, 3, activation='relu'))
model.add(Dropout(0.70))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Dropout(0.70))
model.add(BatchNormalization())
model.add(Conv1D(48, 3, activation='relu'))
model.add(Flatten())

model.add(Dense(6, activation='softmax'))
# model.add(Dropout(0.70))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy']) 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_333 (Conv1D)          (None, 126, 180)          5040      
_________________________________________________________________
dropout_216 (Dropout)        (None, 126, 180)          0         
_________________________________________________________________
conv1d_334 (Conv1D)          (None, 124, 128)          69248     
_________________________________________________________________
dropout_217 (Dropout)        (None, 124, 128)          0         
_________________________________________________________________
conv1d_335 (Conv1D)          (None, 122, 100)          38500     
_________________________________________________________________
dropout_218 (Dropout)        (None, 122, 100)          0         
_________________________________________________________________
conv1d_336 (Conv1D)          (None, 120, 64)           19264     
__________

In [191]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=30)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 119s 16ms/step - loss: 1.6252 - acc: 0.3400 - val_loss: 1.9785 - val_acc: 0.3325
Epoch 2/30
7352/7352 [==============================] - 104s 14ms/step - loss: 0.8570 - acc: 0.5813 - val_loss: 0.9543 - val_acc: 0.5969
Epoch 3/30
7352/7352 [==============================] - 105s 14ms/step - loss: 0.6203 - acc: 0.7160 - val_loss: 0.9646 - val_acc: 0.6016
Epoch 4/30
7352/7352 [==============================] - 104s 14ms/step - loss: 0.4261 - acc: 0.8317 - val_loss: 1.1107 - val_acc: 0.6600
Epoch 5/30
7352/7352 [==============================] - 105s 14ms/step - loss: 0.3186 - acc: 0.8838 - val_loss: 0.8718 - val_acc: 0.7533
Epoch 6/30
7352/7352 [==============================] - 105s 14ms/step - loss: 0.2743 - acc: 0.8979 - val_loss: 1.0316 - val_acc: 0.7357
Epoch 7/30
7352/7352 [==============================] - 105s 14ms/step - loss: 0.2301 - acc: 0.9123 - val_loss: 0.7011 - val_acc: 

In [192]:
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  0      440        40        0                   3   
STANDING                 0      136       394        0                   0   
WALKING                  0        1         0      425                  58   
WALKING_DOWNSTAIRS       0        0         0        0                 420   
WALKING_UPSTAIRS         0        2         0        0                  31   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            8  
STANDING                           2  
WALKING                           12  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 438  


In [193]:
model = Sequential()
model.add(Conv1D(180, 3, input_shape=(timesteps, input_dim), activation='relu'))
model.add(Dropout(0.70))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Dropout(0.70))
model.add(Conv1D(100, 3, activation='relu'))
model.add(Dropout(0.70))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Dropout(0.70))
model.add(BatchNormalization())
model.add(Conv1D(48, 3, activation='relu'))
model.add(Flatten())

model.add(Dense(6, activation='softmax'))
# model.add(Dropout(0.70))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy']) 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_338 (Conv1D)          (None, 126, 180)          5040      
_________________________________________________________________
dropout_220 (Dropout)        (None, 126, 180)          0         
_________________________________________________________________
conv1d_339 (Conv1D)          (None, 124, 128)          69248     
_________________________________________________________________
dropout_221 (Dropout)        (None, 124, 128)          0         
_________________________________________________________________
conv1d_340 (Conv1D)          (None, 122, 100)          38500     
_________________________________________________________________
dropout_222 (Dropout)        (None, 122, 100)          0         
_________________________________________________________________
conv1d_341 (Conv1D)          (None, 120, 64)           19264     
__________

In [194]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=30)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 125s 17ms/step - loss: 1.1485 - acc: 0.5635 - val_loss: 4.5156 - val_acc: 0.3739
Epoch 2/30
7352/7352 [==============================] - 110s 15ms/step - loss: 0.6023 - acc: 0.7745 - val_loss: 2.3681 - val_acc: 0.4808
Epoch 3/30
7352/7352 [==============================] - 110s 15ms/step - loss: 0.4101 - acc: 0.8444 - val_loss: 1.2354 - val_acc: 0.6376
Epoch 4/30
7352/7352 [==============================] - 109s 15ms/step - loss: 0.3293 - acc: 0.8720 - val_loss: 0.9711 - val_acc: 0.6922
Epoch 5/30
7352/7352 [==============================] - 109s 15ms/step - loss: 0.2608 - acc: 0.9032 - val_loss: 0.9502 - val_acc: 0.6780
Epoch 6/30
7352/7352 [==============================] - 109s 15ms/step - loss: 0.2068 - acc: 0.9189 - val_loss: 1.2295 - val_acc: 0.7000
Epoch 7/30
7352/7352 [==============================] - 110s 15ms/step - loss: 0.1975 - acc: 0.9248 - val_loss: 0.7095 - val_acc: 

In [195]:
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  0      430        59        0                   0   
STANDING                 0      174       358        0                   0   
WALKING                  0        0         1      473                  22   
WALKING_DOWNSTAIRS       0        0         0       13                 407   
WALKING_UPSTAIRS         0        0         1       35                  24   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            2  
STANDING                           0  
WALKING                            0  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 411  


In [196]:
model = Sequential()
model.add(Conv1D(180, 3, input_shape=(timesteps, input_dim), activation='sigmoid'))
model.add(Dropout(0.70))
model.add(Conv1D(128, 3, activation='sigmoid'))
model.add(Dropout(0.70))
model.add(Conv1D(100, 3, activation='sigmoid'))
model.add(Dropout(0.70))
model.add(Conv1D(64, 3, activation='sigmoid'))
model.add(Dropout(0.70))
model.add(BatchNormalization())
model.add(Conv1D(48, 3, activation='sigmoid'))
model.add(Flatten())

model.add(Dense(6, activation='softmax'))
# model.add(Dropout(0.70))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy']) 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_343 (Conv1D)          (None, 126, 180)          5040      
_________________________________________________________________
dropout_224 (Dropout)        (None, 126, 180)          0         
_________________________________________________________________
conv1d_344 (Conv1D)          (None, 124, 128)          69248     
_________________________________________________________________
dropout_225 (Dropout)        (None, 124, 128)          0         
_________________________________________________________________
conv1d_345 (Conv1D)          (None, 122, 100)          38500     
_________________________________________________________________
dropout_226 (Dropout)        (None, 122, 100)          0         
_________________________________________________________________
conv1d_346 (Conv1D)          (None, 120, 64)           19264     
__________

In [197]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=30)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 81s 11ms/step - loss: 2.0911 - acc: 0.1782 - val_loss: 2.0241 - val_acc: 0.1598
Epoch 2/30
7352/7352 [==============================] - 72s 10ms/step - loss: 1.7430 - acc: 0.2572 - val_loss: 1.3453 - val_acc: 0.4147
Epoch 3/30
7352/7352 [==============================] - 71s 10ms/step - loss: 1.1830 - acc: 0.4743 - val_loss: 1.3303 - val_acc: 0.4778
Epoch 4/30
7352/7352 [==============================] - 70s 10ms/step - loss: 1.1033 - acc: 0.5146 - val_loss: 1.3857 - val_acc: 0.4720
Epoch 5/30
7352/7352 [==============================] - 72s 10ms/step - loss: 0.8494 - acc: 0.6662 - val_loss: 0.9298 - val_acc: 0.6390
Epoch 6/30
7352/7352 [==============================] - 71s 10ms/step - loss: 0.5539 - acc: 0.7644 - val_loss: 0.7508 - val_acc: 0.6885
Epoch 7/30
7352/7352 [==============================] - 73s 10ms/step - loss: 0.4860 - acc: 0.7939 - val_loss: 0.4777 - val_acc: 0.8144


In [198]:
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0        0                   0   
SITTING                  0      340       151        0                   0   
STANDING                 0       47       485        0                   0   
WALKING                  0        0         2      466                  12   
WALKING_DOWNSTAIRS       0        0         0        0                 397   
WALKING_UPSTAIRS         0        0         0        6                  17   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                            0  
STANDING                           0  
WALKING                           16  
WALKING_DOWNSTAIRS                23  
WALKING_UPSTAIRS                 448  


In [199]:
# https://github.com/heeryoncho/sensors2018cnnhar/blob/master/har/har_dyna_learn_model.py
model = Sequential()
model.add(Conv1D(100, 3, input_shape=(timesteps, input_dim), activation='relu'))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(6, activation='softmax'))
model.add(Dropout(0.5))

adam = Adam(lr=0.0004, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='mean_squared_error', optimizer=adam, metrics=['accuracy'])

# Summarize layers
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_348 (Conv1D)          (None, 126, 100)          2800      
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 42, 100)           0         
_________________________________________________________________
flatten_56 (Flatten)         (None, 4200)              0         
_________________________________________________________________
dense_72 (Dense)             (None, 6)                 25206     
_________________________________________________________________
dropout_228 (Dropout)        (None, 6)                 0         
Total params: 28,006
Trainable params: 28,006
Non-trainable params: 0
_________________________________________________________________
None


In [200]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=30)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 39s 5ms/step - loss: 0.1290 - acc: 0.4241 - val_loss: 0.0736 - val_acc: 0.7950
Epoch 2/30
7352/7352 [==============================] - 27s 4ms/step - loss: 0.1158 - acc: 0.4708 - val_loss: 0.0656 - val_acc: 0.8548
Epoch 3/30
7352/7352 [==============================] - 27s 4ms/step - loss: 0.1104 - acc: 0.4833 - val_loss: 0.0609 - val_acc: 0.8846
Epoch 4/30
7352/7352 [==============================] - 27s 4ms/step - loss: 0.1092 - acc: 0.4848 - val_loss: 0.0584 - val_acc: 0.8870
Epoch 5/30
7352/7352 [==============================] - 26s 3ms/step - loss: 0.1074 - acc: 0.4895 - val_loss: 0.0590 - val_acc: 0.8911s - loss: 0.1084 - acc: 0.485 - ETA: 9s - l - ETA: 7s - loss: 0.108 - ETA: 6s - loss: 0.1 - ETA: 3s - loss: 0.1077 - acc: 0.487 - ETA: 3s - loss: 0.1076 - a - ETA: 0s - loss: 0.1074 - a
Epoch 6/30
7352/7352 [==============================] - 24s 3ms/step - loss: 0.1073 - acc: 0

In [201]:
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  0      400        69        0                   0   
STANDING                 0      102       430        0                   0   
WALKING                  0        0         0      479                  16   
WALKING_DOWNSTAIRS       0        2         0        2                 414   
WALKING_UPSTAIRS         0        0         0       11                  24   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                           22  
STANDING                           0  
WALKING                            1  
WALKING_DOWNSTAIRS                 2  
WALKING_UPSTAIRS                 436  


In [202]:
model = Sequential()
model.add(Conv1D(30, 3, input_shape=(timesteps, input_dim), activation='relu'))
model.add(Conv1D(50, 3, activation='relu'))
model.add(Conv1D(100, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(6, activation='softmax'))
model.add(Dropout(0.50))

adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='mean_squared_error', optimizer=adam, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_349 (Conv1D)          (None, 126, 30)           840       
_________________________________________________________________
conv1d_350 (Conv1D)          (None, 124, 50)           4550      
_________________________________________________________________
conv1d_351 (Conv1D)          (None, 122, 100)          15100     
_________________________________________________________________
flatten_57 (Flatten)         (None, 12200)             0         
_________________________________________________________________
dense_73 (Dense)             (None, 6)                 73206     
_________________________________________________________________
dropout_229 (Dropout)        (None, 6)                 0         
Total params: 93,696
Trainable params: 93,696
Non-trainable params: 0
_________________________________________________________________
None


In [203]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=30)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 31s 4ms/step - loss: 0.1336 - acc: 0.4051 - val_loss: 0.0837 - val_acc: 0.6498
Epoch 2/30
7352/7352 [==============================] - 18s 2ms/step - loss: 0.1217 - acc: 0.4517 - val_loss: 0.0706 - val_acc: 0.8269
Epoch 3/30
7352/7352 [==============================] - 18s 2ms/step - loss: 0.1110 - acc: 0.4879 - val_loss: 0.0659 - val_acc: 0.8673
Epoch 4/30
7352/7352 [==============================] - 18s 2ms/step - loss: 0.1087 - acc: 0.4853 - val_loss: 0.0629 - val_acc: 0.8724
Epoch 5/30
7352/7352 [==============================] - 18s 2ms/step - loss: 0.1065 - acc: 0.4947 - val_loss: 0.0635 - val_acc: 0.8741
Epoch 6/30
7352/7352 [==============================] - 17s 2ms/step - loss: 0.1065 - acc: 0.4878 - val_loss: 0.0613 - val_acc: 0.8850
Epoch 7/30
7352/7352 [==============================] - 18s 2ms/step - loss: 0.1070 - acc: 0.4838 - val_loss: 0.0603 - val_acc: 0.8880
Epoch 8

In [204]:
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0        0                   0   
SITTING                  0      415        49        0                   0   
STANDING                 0       75       455        0                   0   
WALKING                  0        1         0      477                  12   
WALKING_DOWNSTAIRS       0        0         0        1                 415   
WALKING_UPSTAIRS         0        0         0       12                  22   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                           27  
STANDING                           2  
WALKING                            6  
WALKING_DOWNSTAIRS                 4  
WALKING_UPSTAIRS                 437  


In [206]:
# https://github.com/heeryoncho/sensors2018cnnhar/blob/master/har/har_dyna_learn_model.py
model = Sequential()
model.add(Conv1D(100, 3, input_shape=(timesteps, input_dim), activation='relu'))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(6, activation='softmax'))
model.add(Dropout(0.5))

adam = Adam(lr=0.0004, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

# Summarize layers
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_353 (Conv1D)          (None, 126, 100)          2800      
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 42, 100)           0         
_________________________________________________________________
flatten_59 (Flatten)         (None, 4200)              0         
_________________________________________________________________
dense_75 (Dense)             (None, 6)                 25206     
_________________________________________________________________
dropout_231 (Dropout)        (None, 6)                 0         
Total params: 28,006
Trainable params: 28,006
Non-trainable params: 0
_________________________________________________________________
None


In [207]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=30)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 41s 6ms/step - loss: 1.5143 - acc: 0.8411 - val_loss: 0.2502 - val_acc: 0.8933
Epoch 2/30
7352/7352 [==============================] - 26s 4ms/step - loss: 1.5138 - acc: 0.8412 - val_loss: 0.1998 - val_acc: 0.9155
Epoch 3/30
7352/7352 [==============================] - 27s 4ms/step - loss: 1.5682 - acc: 0.8396 - val_loss: 0.3520 - val_acc: 0.9094 1.5105 - acc: 0.84 - ETA: 5s - loss: 1.509 - ETA: 2s - loss: 1.548 - ETA: 1s - loss: 1.556 - ETA: 0s - loss: 1.5610 - acc: 0
Epoch 4/30
7352/7352 [==============================] - 26s 3ms/step - loss: 1.6205 - acc: 0.8273 - val_loss: 0.2298 - val_acc: 0.9117
Epoch 5/30
7352/7352 [==============================] - 26s 3ms/step - loss: 1.5085 - acc: 0.8448 - val_loss: 0.2064 - val_acc: 0.9188
Epoch 6/30
7352/7352 [==============================] - 26s 3ms/step - loss: 1.5108 - acc: 0.8403 - val_loss: 0.2658 - val_acc: 0.8953
Epoch 7/30
7352/7

In [208]:
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0        0                   0   
SITTING                  0      409        80        0                   0   
STANDING                 0      100       430        0                   0   
WALKING                  0        0         0      428                  49   
WALKING_DOWNSTAIRS       0        0         0       12                 397   
WALKING_UPSTAIRS         0        0         0        7                  35   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                            2  
STANDING                           2  
WALKING                           19  
WALKING_DOWNSTAIRS                11  
WALKING_UPSTAIRS                 429  
